In [1]:
from sklearn.externals import joblib

In [2]:
clf = joblib.load('classify_caffe.model') # 분류기 엔진
cate_dict = joblib.load('cate_dict_caffe.dat') # id별 카테고리가 맵핑되어있는 파일
vectorizer = joblib.load('vectorizer_caffe.dat') # 제목들을 숫자 id로 변환해주는 것

In [3]:
cate_id_name_dict = dict(map(lambda (k,v):(v,k),cate_dict.items()))

In [ ]:
# initialize size
size = 0
joblib.dump(size,'size.dat')

In [9]:
# read json results of samples
list_caffe_samples = joblib.load('list_caffe_samples.dat')
'''
res = list_caffe_samples[size]['result']
    if res[0] :
        for each in res[2] :
            name += " " + each[0].replace(" ", "")
'''

[True, [[u'web site', u'0.56633'], [u'Band Aid', u'0.12045'], [u'packet', u'0.04885'], [u'cash machine', u'0.04730'], [u'envelope', u'0.04477']], [[u'web site', u'2.22455'], [u'computer', u'1.63205'], [u'machine', u'1.38676'], [u'device', u'0.50383'], [u'container', u'0.17822']], u'5.054']


In [ ]:
from bottle import route, run, template,request,get, post


import  time
from threading import  Condition
_CONDITION = Condition()

@route('/classify')
def classify():
    print "classify called"
    img = request.GET.get('img','')
    name = request.GET.get('name', '')
    
    # read size
    size = joblib.load('size.dat')
    
    # append caffe result
    res = list_caffe_samples[size]['result']
    if res[0] :
        for each in res[2] :
            name += " " + each[0].replace(" ", "")
    
    vec = vectorizer.transform([name])
    p = clf.predict(vec)
    pred = p[0]
    
    # update and store size
    size = size + 1;
    joblib.dump(size,'size.dat')
    
    return {'cate':cate_id_name_dict[pred]}


run(host='0.0.0.0', port=8887)
# http://192.168.99.100:8887/classify?name=%27ad%27&img=%27asd%27

Bottle v0.12.9 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8887/
Hit Ctrl-C to quit.



classify called
vec
  (0, 22991)	1
p
[6]


192.168.99.1 - - [15/Aug/2016 10:31:49] "GET /classify?name=%EC%B6%95%EA%B5%AC&img=%27asd%27 HTTP/1.1" 200 80
192.168.99.1 - - [15/Aug/2016 10:31:49] "GET /favicon.ico HTTP/1.1" 404 747


classify called
vec
  (0, 265)	1
  (0, 18173)	1
  (0, 19597)	1
  (0, 19786)	1
  (0, 20629)	1
  (0, 20758)	1
  (0, 22418)	1
  (0, 25237)	1
p
[33]


172.17.0.1 - - [15/Aug/2016 10:32:05] "GET /classify?name=[%EC%8B%A0%ED%95%9C%EC%B9%B4%EB%93%9C5%%ED%95%A0%EC%9D%B8][%EC%98%88%ED%99%94-%EC%A2%8B%EC%9D%80%EC%95%84%EC%9D%B4%EB%93%A4]%20%EC%95%84%EB%8F%99%ED%95%9C%EB%B3%B5%20%EC%97%AC%EC%95%84%201076%20%EB%B9%9B%EC%9D%B4%EB%82%98%EB%85%B8%EB%9E%91&img= HTTP/1.1" 200 74


 * 추후 여기 docker 에서 뭔가 python package 설치할게 있으면 
 * /opt/conda/bin/pip2 install bottle 이런식으로 설치 가능